In [130]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

df=pd.read_excel('../dataProcessing/outputOfJoinedTablexceptReleaseDate19862023.xlsx')

In [134]:
df.dropna(subset=['Genre_metacritic_game_info'], inplace=True)
df.drop('releasedate_link',axis='columns', inplace=True)
df.info ()
#print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 826 entries, 21 to 8874
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   title                                826 non-null    object 
 1   platform                             826 non-null    object 
 2   genre                                826 non-null    object 
 3   publisher                            826 non-null    object 
 4   vg_score                             826 non-null    float64
 5   critic_score                         826 non-null    float64
 6   user_score                           826 non-null    float64
 7   total_shipped                        826 non-null    float64
 8   aisan_sales                          826 non-null    float64
 9   north_american_sales                 826 non-null    float64
 10  japan_sales                          826 non-null    float64
 11  european_sales                

Data pre-processing

In [135]:
df['years_since_release'] = 2023 - df['release_year']
df['avg_sales_since_release'] = df['global_sales'] / df['years_since_release']

df['Net_profit']=df['global_sales']-df['Production Cost']

df['Profit_Hit_Miss']=np.where(df['Net_profit'] >= 0, 'hit', 'miss')
#df.to_excel('new_df.xlsx', index=False)
df

,title,platform,genre,publisher,vg_score,critic_score,user_score,total_shipped,aisan_sales,north_american_sales,...,Avg_Userscore_metacritic_game_info,No. of players_metacritic_game_info,releasedate_1986-2023,releasedate_VGOutput,releasedate_raw,releasedate_quarter,years_since_release,avg_sales_since_release,Net_profit,Profit_Hit_Miss
21,1701 A.D.,PC,Simulation,Sony Computer Entertainment,8.0,7.7,5.0,1.00,7.05,4.74,...,8.4,1 to 4,NaN,NaN,October 26 2006,2006-10-26 00:00:00,17,1.172941,12.98,hit
40,2002 FIFA World Cup,PC,Sports,Sony Computer Entertainment,2.0,6.0,7.0,277.00,3.77,3.55,...,8,Mutliplayer,NaN,NaN,2002-04-26 00:00:00,2002-04-26 00:00:00,21,0.568095,5.87,hit
42,2010 FIFA World Cup South Africa,PS3,Sports,Ubisoft,7.0,8.2,9.0,53.00,1.23,0.30,...,7.8,1,NaN,NaN,April 27 2010,2010-04-27 00:00:00,13,0.172308,-2.00,miss
114,ABZU,PC,Adventure,Activision,9.0,8.0,9.0,0.58,6.83,1.60,...,6.9,1,NaN,NaN,August 2 2016,2016-08-02 00:00:00,7,2.341429,12.64,hit
115,ABZU,PS4,Adventure,Electronic Arts,8.0,7.0,7.0,117.00,3.00,0.03,...,7.6,1,NaN,NaN,August 2 2016,2016-08-02 00:00:00,6,1.505000,5.79,hit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8847,Zeno Clash,PC,Shooter,Activision,8.4,7.0,2.0,0.32,1.93,1.74,...,7.9,1,NaN,NaN,2009-04-21 00:00:00,2009-04-21 00:00:00,14,0.648571,5.06,hit
8851,Zero Escape Zero Time Dilemma,PC,Adventure,Activision,10.0,7.0,7.0,67.00,7.57,1.50,...,8.3,1,NaN,NaN,June 30 2016,2016-06-30 00:00:00,7,2.782857,17.99,hit
8852,Zero Escape Zero Time Dilemma,PS4,Adventure,Omega,10.0,10.0,10.0,133.00,3.00,0.02,...,7.2,1,NaN,NaN,August 17 2017,2017-08-17 00:00:00,6,0.946667,0.37,hit
8873,Zone of the Enders,PS2,Simulation,IBM,9.0,7.5,8.0,52.00,3.00,0.30,...,7.6,1 to 2,NaN,NaN,March 23 2001,2001-03-23 00:00:00,22,0.064545,1.38,hit


In [136]:
#Encoding 
encoder = OneHotEncoder(sparse=False)

#Genre
genre_encoded = pd.get_dummies(df['genre'], prefix='genre', drop_first=True)
#data.drop('genre', axis=1, inplace=True) 

#Platform
platform_encoded = pd.get_dummies(df['platform'], prefix='platform', drop_first=True)

#Publisher
publisher_encoded = pd.get_dummies(df['publisher'], prefix='publisher', drop_first=True)

#Players
Players_encoded = pd.get_dummies(df['No. of players_metacritic_game_info'], prefix='Players', drop_first=True)

In [137]:
data = pd.concat([df, genre_encoded, publisher_encoded,publisher_encoded,Players_encoded], axis=1)
data.to_excel('test_onehot1.xlsx', index=False)

In [126]:
features = ['critic_score', 'user_score', 'Production Cost']

# Input variables redefine
X = pd.concat([genre_encoded, df[features]], axis=1)
y = data['avg_sales_since_release']

X

,genre_Board Game,genre_Fighting,genre_Puzzle,genre_Racing,genre_Shooter,genre_Simulation,genre_Sports,critic_score,user_score,Production Cost
21,0,0,0,0,0,1,0,7.7,5.0,6.96
40,0,0,0,0,0,0,1,6.0,7.0,6.06
42,0,0,0,0,0,0,1,8.2,9.0,4.24
114,0,0,0,0,0,0,0,8.0,9.0,3.75
115,0,0,0,0,0,0,0,7.0,7.0,3.24
...,...,...,...,...,...,...,...,...,...,...
8847,0,0,0,0,1,0,0,7.0,2.0,4.02
8851,0,0,0,0,0,0,0,7.0,7.0,1.49
8852,0,0,0,0,0,0,0,10.0,10.0,5.31
8873,0,0,0,0,0,1,0,7.5,8.0,0.04


In [125]:
#Data splitting
# Split into training and test sets (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50, shuffle=True)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

Training set: 578 samples
Testing set: 248 samples


Model implementation

In [110]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


x1_train, x1_test, y_train, y_test = train_test_split(data[features], y, test_size=0.2, random_state=50, shuffle=True)

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(x1_train, y_train)
lr_predictions = lr_model.predict(x1_test)

lr_mse = mean_squared_error(y_test, lr_predictions)
lr_mae = mean_absolute_error(y_test, lr_predictions)
lr_rmse = np.sqrt(lr_mse)
lr_r2 = r2_score(y_test, lr_predictions)

print(f"Linear Regression Mean Squared Error (MSE): {lr_mse}")
print(f"Linear Regression Mean Absolute Error (MAE): {lr_mae}")
print(f"Linear Regression Root-mean-square deviation (RMSE): {lr_rmse}")
print(f"Linear Regression R-squared (R2): {lr_r2}")

Linear Regression Mean Squared Error (MSE): 0.44047707603255715
Linear Regression Mean Absolute Error (MAE): 0.45532190362335334
Linear Regression Root-mean-square deviation (RMSE): 0.6636844702360882
Linear Regression R-squared (R2): -0.020609402705795254


In [112]:
# Neural Network
model = keras.Sequential([
    keras.layers.Dense(10, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(1)  # Linear output for regression
])

model.compile(optimizer='sgd', loss='mean_squared_error', metrics = "accuracy")
model.fit(X_train, y_train, epochs=100, batch_size=100, verbose=1)

nn_predictions = model.predict(X_test)
nn_mse = mean_squared_error(y_test, nn_predictions)
nn_mae = mean_absolute_error(y_test, nn_predictions)
nn_r2 = r2_score(y_test, nn_predictions)

print(f"Neural Network Mean Squared Error (MSE): {nn_mse}")
print(f"Neural Network Mean Absolute Error (MAE): {nn_mae}")
print(f"Neural Network R-squared (R2): {nn_r2}")


Epoch 1/100
7/7 [==============================] - 0s 1ms/step - loss: 2.1558 - accuracy: 0.0015
Epoch 2/100
7/7 [==============================] - 0s 825us/step - loss: 0.4862 - accuracy: 0.0015
Epoch 3/100
7/7 [==============================] - 0s 831us/step - loss: 0.4525 - accuracy: 0.0015
Epoch 4/100
7/7 [==============================] - 0s 997us/step - loss: 0.4406 - accuracy: 0.0015
Epoch 5/100
7/7 [==============================] - 0s 831us/step - loss: 0.4246 - accuracy: 0.0015
Epoch 6/100
7/7 [==============================] - 0s 831us/step - loss: 0.4151 - accuracy: 0.0015
Epoch 7/100
7/7 [==============================] - 0s 831us/step - loss: 0.4119 - accuracy: 0.0015
Epoch 8/100
7/7 [==============================] - 0s 673us/step - loss: 0.4094 - accuracy: 0.0015
Epoch 9/100
7/7 [==============================] - 0s 805us/step - loss: 0.4086 - accuracy: 0.0015
Epoch 10/100
7/7 [==============================] - 0s 831us/step - loss: 0.4076 - accuracy: 0.0015
Epoch 11/10

In [86]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

print(f"Random Forest Mean Squared Error (MSE): {rf_mse}")
print(f"Random Forest Mean Absolute Error (MAE): {rf_mae}")
print(f"Random Forest R-squared (R2): {rf_r2}")

Random Forest Mean Squared Error (MSE): 40.2053547076506
Random Forest Mean Absolute Error (MAE): 5.196784939759037
Random Forest R-squared (R2): -0.06994124151139203


Hit or Miss prediction

In [139]:
features = ['critic_score', 'user_score', 'Production Cost']

# Input variables redefine
X = pd.concat([genre_encoded,publisher_encoded,platform_encoded, df[features]], axis=1)
y = data['Profit_Hit_Miss']

#Data splitting
# Split into training and test sets (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50, shuffle=True)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

Training set: 660 samples
Testing set: 166 samples


In [141]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_accuracy

0.7530120481927711

In [142]:
# Neural Network for Binary Classification
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  
])

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, (y_train == 'hit').astype(int), epochs=100, batch_size=32, verbose=0)

nn_predictions = model.predict(X_test)
nn_predictions = (nn_predictions > 0.5)  

nn_accuracy = accuracy_score((y_test == 'hit').astype(int), nn_predictions)
nn_accuracy

6/6 [==============================] - 0s 797us/step


0.8012048192771084